In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
shelter = AnimalShelter(
    user="aacuser",
    password="potato",
    host="nv-desktop-services.apporto.com",
    port=30702,
    db="AAC",
    col="animals"
)

# Connect to database via CRUD Module
records = shelter.read({})
if records is None:
    records = []
if not isinstance(records, list):
    raise TypeError("AnimalShelter.read() must return a list of documents.")
df = pd.DataFrame.from_records(records)

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'],inplace=True)


## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Chris Bridges')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),

app.layout = html.Div([
     html.Div([
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={'height': '70px', 'margin': '10px'}
        ),
        html.H4("Module 6 Assignment: Chris Bridges", style={'margin': '10px 0 0 20px'})
    ], style={'display': 'flex', 'align-items': 'center', 'margin-bottom': '5px'}),

    html.Hr(style={'margin-top': '5px', 'margin-bottom': '10px'}),

    html.Div(  
    dcc.RadioItems(
        id='rescue-type-filter',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',
        labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        ),
    ),
    html.Hr(),
     dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        row_selectable="single",
        selected_rows=[0],
        page_size=10,
        sort_action='native',
        filter_action='native',
        style_table={'overflowX': 'auto'},
        style_cell={
            'textAlign': 'left',
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal'
        }
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
    ])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id', 'data'),
              [Input('rescue-type-filter', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'Water Rescue':
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland Mix"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'Disaster or Individual Tracking':
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        query = {}

    filtered_records = shelter.read(query)
    if not filtered_records:
        return []
    
    df_filtered = pd.DataFrame.from_records(filtered_records)
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)
    return df_filtered.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return html.Div("No data available for chart.")
    
    dff = pd.DataFrame(viewData)
    if 'breed' not in dff.columns:
        return html.Div("Breed column not found in data.")

    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return dcc.Graph(figure=fig)
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    # Returns a default message if no data is available
    if viewData is None or len(viewData) == 0:
        return [html.Div("No data available to display on the map.")]
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    row = 0
    if index is not None and len(index) > 0:
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    lat = dff.iloc[row].get("location_lat", None)
    lon = dff.iloc[row].get("location_long", None)
    if lat is None or lon is None:
        return [html.Div("Missing latitude or longitude data for selected animal.")]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
            center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row].get("breed", "Unknown")),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row].get("name", "Unknown"))
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:27512/
